# Install and import requirements

In [ ]:
# Install required libraries
!pip install datasets transformers seqeval scipy tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=f9207db4963e7b30dc133c48083e3b905104b0a8896c11f25b2278a4270bedc5
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency 

In [ ]:
import json

import pandas as pd

import torch

from scipy.stats import spearmanr

from tqdm import tqdm

import numpy as np

from collections import Counter

from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForTokenClassification, EarlyStoppingCallback

from datasets import Dataset

import re

# Upload data

*  validation data - 'mushroom.en-val.v2.jsonl'
*  sample label files - 'en_train_labeled.jsonl'

In [ ]:
# Upload validation data and sample label files
from google.colab import files
uploaded = files.upload()  # Upload your dataset files

Saving mushroom.en-val.v2.jsonl to mushroom.en-val.v2.jsonl


In [ ]:
import json

def load_jsonl_file_to_records(filename):
    records = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                records.append(json.loads(line))

        return pd.DataFrame(records)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

In [ ]:
train_df = load_jsonl_file_to_records('en_train_labeled.jsonl')
#train_df2 = pd.read_csv("test_out.csv")

test_df = load_jsonl_file_to_records('mushroom.en-val.v2.jsonl')

## Remove responses with "I'm sorry" form train data

In [ ]:
print("Length of train file:", len(train_df))
print("Length of validation file:", len(test_df))

Length of train file: 809
Length of validation file: 50


In [ ]:
# Define the regex pattern to match "I'm sorry"
pattern = re.compile(r"I'm sorry")

# Apply the pattern match to the train_df
train_df["model_output_text_match"] = train_df["model_output_text"].apply(lambda x: bool(pattern.search(x)))

# Filter out the expressions that contain the pattern
train_df = train_df[~train_df["model_output_text_match"]]

# Drop the helper column used for matching
train_df = train_df.drop(columns=["model_output_text_match"])

In [ ]:
print("Length of train file:", len(train_df))
print("Length of validation file:", len(test_df))

Length of train file: 762
Length of validation file: 50


## Only leave model input, output text and hard labels in train data

In [ ]:
train_df = train_df[["model_input", "model_output_text", "hard_labels"]]

In [ ]:
train_df.head()

,model_input,model_output_text,hard_labels
0,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[[82, 87], [22, 31], [52, 61], [10, 17], [43, ..."
1,Do all arthropods have antennae?,"Yes, all insects and arachnids have at least ...","[[40, 45], [22, 31], [10, 17], [50, 57], [46, ..."
2,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[[82, 87], [22, 31], [52, 61], [10, 17], [43, ..."
3,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[[52, 61], [22, 31], [10, 17], [43, 50], [82, ..."
4,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[[82, 87], [22, 31], [67, 72], [52, 61], [10, ..."


In [ ]:
test_df.head()

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels,model_output_logits,model_output_tokens
0,val-en-1,EN,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,tiiuae/falcon-7b-instruct,"[{'start': 10, 'prob': 0.2, 'end': 12}, {'star...","[[25, 31], [45, 49], [69, 83]]","[-5.5669536591, -11.90533638, -13.0743436813, ...","[Pet, ra, Ġvan, ĠSto, ve, ren, Ġwon, Ġa, Ġsilv..."
1,val-en-2,EN,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,tiiuae/falcon-7b-instruct,"[{'start': 4, 'prob': 0.2, 'end': 14}, {'start...","[[30, 31]]","[-6.199614048, -13.7564926147, -14.0058326721,...","[The, ĠE, lys, iph, ale, Ġorder, Ġcontains, Ġ,..."
2,val-en-3,EN,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.6, 'end': 3}, {'start'...","[[0, 3], [9, 18], [63, 70], [78, 87]]","[-4.8190689087, -16.5279369354, -10.1344690323...","[Yes, ,, Ġall, Ġar, ach, n, ids, Ġhave, Ġanten..."
3,val-en-4,EN,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,tiiuae/falcon-7b-instruct,"[{'start': 18, 'prob': 0.0909090909, 'end': 29...","[[29, 33]]","[-6.2555613518, -10.9392805099, -13.1335763931...","[Ch, ance, Ġthe, Ġrapper, Ġdebuted, Ġin, Ġ, 20..."
4,val-en-5,EN,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.0909090909, 'end': 9},...","[[70, 227]]","[-3.9737114906, -12.9197320938, -7.2846975327,...","[The, ĠUN, ', s, ĠSustainable, ĠCity, Ġinitiat..."


# Train

In [ ]:
from transformers import BertForTokenClassification, AutoTokenizer, AutoModelForTokenClassification

# Tokenizer & Model Setup
model_checkpoint = "roberta-base"
tokenizer =  AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)


# Convert Pandas DataFrame to Huggingface Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenize and align labels

In [ ]:
def tokenize_and_align_labels(example):
    """
    Tokenize and align word-level labels with subword tokens for a combined input-output scenario.
    Parameters:
    example (dict): A dictionary containing:
                     - "model_input": input question.
                     - "model_output_text": generated response.
                     - "hard_labels": list of (start, end) for words in the output.
    Returns:
    tokenized_inputs (dict): Tokenized input with aligned labels.
    """
    concatenated_text = example['model_input'] + example['model_output_text']
    max_length = 256

    tokenized_inputs = tokenizer(
        concatenated_text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_offsets_mapping=True
    )

    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'])
    offsets = tokenized_inputs['offset_mapping']

    aligned_labels = [-100] * len(tokenized_inputs['input_ids'])
    input_length =len(example['model_input'])
    output_start_idx = len(tokenizer(example['model_input'], truncation=True)['input_ids'])
    #print(output_start_idx)

    for i, (start_char, end_char) in enumerate(offsets):
        # Ignore special tokens and input tokens
        if i < output_start_idx -1 or start_char == end_char:
            continue

        label = 0
        for start, end in example['hard_labels']:
            if start_char >= (start + input_length) and end_char <= (end + input_length):
                label = 1
                break
        if i < len(aligned_labels):
            aligned_labels[i] = label

    tokenized_inputs['labels'] = aligned_labels
    tokenized_inputs['input_length'] = input_length
    tokenized_inputs['output_length'] = len(example['model_output_text'])
    tokenized_inputs['offsets'] = offsets

    return tokenized_inputs

In [ ]:
# Tokenize datasets and align labels
train_dataset = train_dataset.map(tokenize_and_align_labels, batched=False)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=False)

Map:   0%|          | 0/762 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import torch.nn.functional as F
from scipy.stats import spearmanr
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def recompute_hard_labels(soft_labels):
    """Optionally, infer hard labels from the soft labels provided"""
    hard_labels = []
    prev_end = -1
    for start, end in (
        (lbl['start'], lbl['end'])
        for lbl in sorted(soft_labels, key=lambda span: (span['start'], span['end']))
        if lbl['prob'] > 0.5
    ):
        if start == prev_end:
            hard_labels[-1][-1] = end
        else:
            hard_labels.append([start, end])
        prev_end = end
    return hard_labels

def score_iou(ref_dict, pred_dict):
    """Computes intersection-over-union between reference and predicted hard labels for a single datapoint."""
    assert ref_dict['id'] == pred_dict['id']
    ref_indices = {idx for span in ref_dict['hard_labels'] for idx in range(*span)}
    pred_indices = {idx for span in pred_dict['hard_labels'] for idx in range(*span)}
    if not pred_indices and not ref_indices: return 1.
    return len(ref_indices & pred_indices) / len(ref_indices | pred_indices)

def score_cor(ref_dict, pred_dict):
    """Computes Spearman correlation between predicted and reference soft labels for a single datapoint."""
    assert ref_dict['id'] == pred_dict['id']
    ref_vec = [0.] * ref_dict['text_len']
    pred_vec = [0.] * ref_dict['text_len']
    for span in ref_dict['soft_labels']:
        for idx in range(span['start'], min(span['end'], ref_dict['text_len'])):
            ref_vec[idx] = span['prob']
    for span in pred_dict['soft_labels']:
        for idx in range(span['start'], min(span['end'], ref_dict['text_len'])):
            pred_vec[idx] = span['prob']
    if len({round(flt, 8) for flt in pred_vec}) == 1 or len({round(flt, 8) for flt in ref_vec}) == 1:
        return float(len({round(flt, 8) for flt in ref_vec}) == len({round(flt, 8) for flt in pred_vec}))
    return spearmanr(ref_vec, pred_vec).correlation

def compute_metrics(p, dataset):
    preds = p.predictions.argmax(-1)  # Get predicted labels (indices)
    logits = p.predictions  # The raw logits for soft label evaluation
    labels = p.label_ids

    # Assuming dataset has input_length, offset_mapping, and output_length
    input_length = dataset['input_length']
    offsets = dataset['offset_mapping']
    output_length = dataset['output_length']

    # Calculate probabilities using softmax
    probabilities = F.softmax(torch.tensor(logits), dim=-1).numpy()
    probabilities = probabilities [:, :, 1]

    # Flatten the predictions and labels (ignore the -100 label used for padding/special tokens)
    prediction_labels = [pred for pred, label in zip(preds.flatten(), labels.flatten()) if label != -100]
    true_labels = [label for label in labels.flatten() if label != -100]
    '''
    # Debugging prints
    print("Logits shape:", logits.shape)
    print("Predictions shape:", preds.shape)
    print("Labels shape:", labels.shape)
    print("Offsets:", offsets)
    print("Input lengths:", input_length)
    print("Output lengths:", output_length)
    '''

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, prediction_labels, average='binary')
    acc = accuracy_score(true_labels, prediction_labels)

    # Prepare reference and predicted soft labels for custom evaluation (IoU/Spearman)
    soft_labels_ref = []
    soft_labels_pred = []

    for i in range(len(preds)):
        ref_dict = {
            'id': i,
            'soft_labels': dataset['soft_labels'][i],
            'hard_labels': dataset['hard_labels'][i],
            'text_len': output_length[i]

        }
        pred_dict = {
            'id': i,
            'soft_labels': [{'start': (s - input_length[i]), 'end': (e - input_length[i]), 'prob': pred_prob}
                            for (pred_prob, (s, e), label) in zip(probabilities[i], offsets[i], labels[i]) if label != -100],
            'hard_labels': recompute_hard_labels([{'start': (s - input_length[i]), 'end': (e - input_length[i]), 'prob': pred_prob}
                                                  for (pred_prob, (s, e), label) in zip(probabilities[i], offsets[i], labels[i]) if label != -100]),
            'text_len': output_length[i]
        }
        soft_labels_ref.append(ref_dict)
        soft_labels_pred.append(pred_dict)
        '''
        # More debugging prints
        print(f"\nExample {i}:")
        print("Predicted soft labels:", pred_dict['soft_labels'])
        print("Reference soft labels:", ref_dict['soft_labels'])
        '''

    # Calculate IoU and Spearman correlation scores
    ious = np.array([score_iou(r, d) for r, d in zip(soft_labels_ref, soft_labels_pred)])
    cors = np.array([score_cor(r, d) for r, d in zip(soft_labels_ref, soft_labels_pred)])

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'iou_mean': f'{ious.mean():.8f} ± {ious.std():.8f}',
        'spearman_mean': f'{cors.mean():.8f} ± {cors.std():.8f}',
    }

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Define Training Arguments with Early Stopping
training_args = TrainingArguments(
    output_dir="./results",           # Directory for model output
    save_strategy="epoch",            # Save model at the end of each epoch
    eval_strategy="epoch",             # Evaluate model at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=18,
    per_device_eval_batch_size=8,
    num_train_epochs=10,              # Number of training epochs
    weight_decay=0.01,                # Regularization
    logging_dir='./logs',             # Directory for logging
    logging_steps=10,
    load_best_model_at_end=True,      # Load the best model at the end
    metric_for_best_model='f1',       # Metric to compare models
    greater_is_better=True,           # Specify if the metric is better when greater
)

# Initialize the Trainer object with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=lambda p: compute_metrics(p, test_dataset),  # Pass dataset to metrics
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

<ipython-input-16-c6e679671179>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Iou Mean,Spearman Mean
1,0.424300,0.830958,0.651972,0.441687,0.351779,0.593333,0.37889449 ± 0.28078806,0.39464398 ± 0.24661274
2,0.298600,1.549557,0.573473,0.428793,0.311044,0.690000,0.37353102 ± 0.27506980,0.37590782 ± 0.25081284
3,0.229400,2.179296,0.539443,0.419873,0.296628,0.718333,0.36560030 ± 0.25680610,0.36564530 ± 0.24681883
4,0.134900,2.312316,0.556071,0.418440,0.300582,0.688333,0.37582657 ± 0.27229692,0.35531270 ± 0.26086543


{'eval_loss': 0.8309577107429504, 'eval_accuracy': 0.6519721577726219, 'eval_f1': 0.44168734491315137, 'eval_precision': 0.35177865612648224, 'eval_recall': 0.5933333333333334, 'eval_iou_mean': '0.37889449 ± 0.28078806', 'eval_spearman_mean': '0.39464398 ± 0.24661274', 'eval_runtime': 0.6828, 'eval_samples_per_second': 73.224, 'eval_steps_per_second': 10.251, 'epoch': 4.0}
